In [ ]:
import json
import asyncio
import pandas as pd

from ragas import SingleTurnSample
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import (
    LLMContextPrecisionWithoutReference,
    Faithfulness,
    ResponseRelevancy,
)

from langchain_community.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings


def load_samples(json_path: str) -> list[SingleTurnSample]:
    """Carga el archivo JSON y lo convierte en objetos SingleTurnSample."""
    with open(json_path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    samples = [
        SingleTurnSample(
            user_input=d["user_input"],
            response=d["response"],
            retrieved_contexts=d["retrieved_contexts"],
            reference=d["reference"],
        )
        for d in raw
    ]
    return samples


def build_components():
    """Instancia el LLM local y los embeddings envueltos para RAGAS."""
    local_llm = Ollama(model="mistral", temperature=0, timeout=60000)
    wrapped_llm = LangchainLLMWrapper(local_llm)

    hf_emb = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    ragas_emb = LangchainEmbeddingsWrapper(hf_emb)

    metrics = {
#        "faithfulness": Faithfulness(llm=wrapped_llm),
        "ctx_precision_no_ref": LLMContextPrecisionWithoutReference(llm=wrapped_llm),

        "answer_relevancy": ResponseRelevancy(llm=wrapped_llm, embeddings=ragas_emb),
    }
    return metrics


async def evaluate(samples, metrics):
    """Calcula las métricas para cada sample de forma asíncrona."""
    results = []
    for s in samples:
        tasks = {name: m.single_turn_ascore(s) for name, m in metrics.items()}
        scores = {name: await coro for name, coro in tasks.items()}
        results.append(scores)
    return results


def main(json_path="Results/ragas_sample_v2.json", output_csv="ragas_metrics_v2.csv"):
    samples = load_samples(json_path)
    metrics = build_components()
    scores = asyncio.run(evaluate(samples, metrics))

    df = pd.DataFrame(scores)
    df.insert(0, "sample_id", range(1, len(samples) + 1))
    print(df)

    df.to_csv(output_csv, index=False)
    print(f"Resultados guardados en {output_csv}")


if __name__ == "__main__":
    main()


   sample_id  ctx_precision_no_ref  answer_relevancy
0          1                   0.0          0.754383
1          2                   1.0          0.750256
2          3                   1.0          0.753680
3          4                   0.0          0.405326
4          5                   1.0          0.683918
5          6                   1.0          0.928188
6          7                   1.0          0.761486
7          8                   1.0          0.813202
8          9                   1.0          0.855925
9         10                   1.0          0.964674
Resultados guardados en ragas_metrics_v1.csv
